## Scikit-learn tutorial: Baseball Analytics in Python 

This is a quick introduction to doing data analysis with Python - namely the two important libraries 
 1. For working with SQL-like data tables use [pandas](https://pandas.pydata.org), and
 2. for statistical modeling use [scikit-learn](http://scikit-learn.org/stable/).

I am essentially copying the nice introduction to scikit-learn that Brad found on [DataCamp](https://www.datacamp.com).  


https://www.datacamp.com/community/tutorials/scikit-learn-tutorial-baseball-1 <br>
https://www.datacamp.com/community/tutorials/scikit-learn-tutorial-baseball-2

I have adapted the first lesson here to use the `pybaseball` module instead of downloanding the data from Sean Lahman's [website.](http://seanlahman.com)

We will be working with a historical data set of MLB team's year over year statistics.  By the end we will have a predictive model that estimates the number of wins a team will have at the end of the season from on-field statistics (hits, ERA, strikeouts, runs, etc...).  Before that we will learn some basics of data prepping and modeling.

### pybaseball

This is a Python library for doing analytics on MLB baseball stats.  The GitHub repository for the library is linked below.

https://github.com/jldbc/pybaseball

The repository provides installation instructions, but essentially **run the next cell.**  After it is finished installing we need to restart this notebook's `kernel` so that library to be available to use.  To do that open the **Kernel** menu above and hitting *'Restart Kernel...'*.

In [ ]:
! pip install pybaseball  #To run a notebook cell hit SHIFT+ENTER

If you want to learn more, feel free to look around the documentation on the GitHub repository linked below.

https://github.com/jldbc/pybaseball/tree/master/docs

Let's get started.

In [ ]:
import pybaseball.lahman as l  # this is how you pull in additional packages

import pandas as pd

Next, we will collect two sets of data.  One called `teams` will have a variety of data about season by season performance of MLB teams (games played, wins, etc...), and `teams_franchises` contains more historical data (for example, if the team is still active today). 

Run the next cell to collect the data.

In [ ]:
teams = l.teams()  # this collects the teams data from Lahman's website

teams_franchises = l.teams_franchises() # this collects the team franchise data

#teams.to_csv("teams.csv")

#teams_franchises.to_csv("teams_franchises.csv")

In [ ]:
#teams = pd.read_csv("teams.csv")

#teams_franchises = pd.read_csv("teams_franchises.csv")

These two tables are stored as what is called a pandas `DataFrame` (think of a SQL table) it has rows and columns of typed data.  

* To see some basic info on `teams` run the next cell.
* To see the first few rows run the cell following.

In [ ]:
teams.info()

Each of the columns contain data related to a specific team and year. Some of the more important variables are listed below. A full list of the variables can be found [here.](http://seanlahman.com/files/database/readme2016.txt)

 * `yearID` - Year
 * `teamID` - Team
 * `franchID` - Franchise (links to TeamsFranchise table)
 * `G` - Games played
 * `W` - Wins
 * `LgWin` - League Champion(Y or N)
 * `WSWin` - World Series Winner (Y or N)
 * `R` - Runs scored
 * `AB` - At bats
 * `H` - Hits by batters
 * `HR` - Homeruns by batters
 * `BB` - Walks by batters
 * `SO` - Strikeouts by batters
 * `SB` - Stolen bases
 * `CS` - Caught stealing
 * `HBP` - Batters hit by pitch
 * `SF` - Sacrifice flies
 * `RA` - Opponents runs scored
 * `ER` - Earned runs allowed
 * `ERA` - Earned run average
 * `CG` - Complete games
 * `SHO` - Shutouts
 * `SV` - Saves
 * `IPOuts` - Outs Pitched (innings pitched x 3)
 * `HA` - Hits allowed
 * `HRA` - Homeruns allowed
 * `BBA` - Walks allowed
 * `SOA` - Strikeouts by pitchers
 * `E` - Errors
 * `DP` - Double Plays
 * `FP` - Fielding percentage
 * `name` - Team’s full name

In [ ]:
teams.head()

##### Placeholder to do some EDA on `teams` to show off pandas a bit.

In [ ]:
import pandas as pd # this is the typical alias for importing pandas

###Note:  I typically keep all of my import statements in the first cell of a notebook.

Pandas is a powerful library, and one of the nice things is that you can do a wide variety of SQL-like operations with data.  For example, the next cell performs an `inner join` on `teams` and `teams_franchises` to create a DataFrame of active teams that have played in seasons with over 150 games. 

In [ ]:
Teams = pd.merge(teams[(teams.G >= 150)], # left_df: a mask to filter teams down to row with over 150 games
                 teams_franchises[(teams_franchises.active == 'Y')], # right_df: a mask to filter just the acitve teams
                 how="inner", # the type of merge or join
                 on="franchID") # the column to match the DataFrames on

Teams.head()

We will start preparing the data so we can eventually run some statistical models on it.  This tutorial shows how to use sklearn to perform two algorithms
1. **K-means clustering** 
2. **Linear Regression**

First let's drop some of the columns that won't be used.

In [ ]:
drop_cols = ['lgID','franchID','divID','Rank','Ghome',
             'L','DivWin','WCWin','LgWin','WSWin','SF',
             'name','park','attendance','BPF','PPF',
             'teamIDBR','teamIDlahman45','teamIDretro',
             'franchID','franchName','active','NAassoc']  # this is a standard Python data type called a list.  


Teams.drop(drop_cols, axis=1, inplace=True) # inplace=True allows us to update the data in one step.

#Note:  
#The default behavior of pandas is to not change the data, 
#so if you are making changes to a DataFrame you will need to 
#use the inplace parameter when available or save the changes in a new variable

### Brief lesson on null or NA values

The `pd.DataFrame.isnull()` method lets you see how many NA values are in a DataFrame.  To see some documentation on this or any python object simply type a `?` after the object you want to learn about.  For example, run the next cell.

In [ ]:
Teams.isnull?

In [ ]:
Teams.isnull().sum(axis=0).tolist() # this prints the number of NA values in each column

We can make this a little more useful in the next cell. I am using a Python `dictionary` constructed using a *list comprehension* method (the embedded `for` statement). 

In [ ]:
{list(Teams.columns)[i] : Teams.isnull().sum(axis=0).tolist()[i] 
 for i in range(len(Teams.columns))}

In [ ]:
# Eliminating columns with a lot null values
Teams = Teams.drop(['CS','HBP'], axis=1)

# Filling null values to the median for the remainer
Teams['SO'] = Teams['SO'].fillna(Teams['SO'].median())
Teams['DP'] = Teams['DP'].fillna(Teams['DP'].median())

# Print out null values of all columns of `df`
print(Teams.isnull().sum(axis=0).tolist())

## Plotting with Matplotlib

There are numerous plotting libraries at your disposal within the python eco-system.  The canonical library is [Matplotlib](https://matplotlib.org).  

In [ ]:
# This allows us to display plots within the notebook
%matplotlib inline  
import matplotlib.pyplot as plt  # the standard alias for matplotlib.

In [ ]:
# Plotting distribution of wins
plt.hist(Teams['W'], bins=30) # method for histograms
plt.xlabel('Wins')
plt.ylabel("Seasons")
plt.title('Distribution of Wins')

plt.show()

In [ ]:
Teams.W.mean()

### How to write a Python function

Python operates with colons and white space to dictate the logical sequence of the code, and not *curly braces* `{}` that are common in other languages.  This might be a challenge to remember at first, but you get the hang of it, and it helps keep the code tidyer and easier to follow.

Template for a function

```python
def fn(param): #colon
    val = param # white space of 4 spaces 
    for i in [1, 2, 3]: #colon
        val = val - i # white space
    return val # note that we reduced white space
```

In [ ]:
# Creating bins for the win column
def assign_win_bins(W): # note the colons (NEEDED AFTER CONTROL FLOW STATEMENTS)
    if W < 50: # <- here
        return 1
    if W >= 50 and W <= 69: # <- here
        return 2
    if W >= 70 and W <= 89: # <- here
        return 3
    if W >= 90 and W <= 109: # <- here
        return 4
    if W >= 110: # <- here
        return 5
        
# Apply `assign_win_bins` to `Teams['W']`    
Teams['win_bins'] = Teams['W'].apply(assign_win_bins)

Teams.head()

In [ ]:
# Plotting scatter graph of Year vs. Wins
plt.scatter(Teams['yearID'], Teams['W'], c=Teams['win_bins'])
plt.title('Wins Scatter Plot')
plt.xlabel('Year')
plt.ylabel('Wins')

plt.show()

In [ ]:
Teams = Teams[Teams.yearID >= 1900]  # this is an example where the data change is not save by default and we save a new variable

### Group by with Pandas

More SQL-like functionality of Pandas.

In [ ]:
Teams.groupby("yearID").W.sum().head()

In [ ]:
year = Teams.groupby("yearID")["R", "G", "HR"].sum() # group by syntax
year.head()

In [ ]:
year = year.assign(mlb_rpg = year.R/year.G) # my prefered syntax for adding a column

year.loc[1904:1910, :] # an alternate way to view the first few rows.  This highlights that yearID is now the index.

In [ ]:
# Create line plot of Year vs. MLB runs per Game
plt.plot(year.index, year.mlb_rpg)
plt.title('MLB Yearly Runs per Game')
plt.xlabel('Year')
plt.ylabel('MLB Runs per Game')

plt.show()

In [ ]:
# Creating "year_label" column, which will give your algorithm information about how certain years are related 
# (Dead ball eras, Live ball/Steroid Eras)

def assign_label(year):
    if year < 1920:
        return 1
    elif year >= 1920 and year <= 1941:
        return 2
    elif year >= 1942 and year <= 1945:
        return 3
    elif year >= 1946 and year <= 1962:
        return 4
    elif year >= 1963 and year <= 1976:
        return 5
    elif year >= 1977 and year <= 1992:
        return 6
    elif year >= 1993 and year <= 2009:
        return 7
    elif year >= 2010:
        return 8
        
# Add `year_label` column to `Teams`    
Teams = Teams.assign(year_label = Teams['yearID'].apply(assign_label))

Teams.head()

In [ ]:
dummy_df = pd.get_dummies(Teams['year_label'], prefix='era')

# Concatenate `Teams` and `dummy_df`
Teams = pd.concat([Teams, dummy_df], axis=1) # the axis=1 statement indicates we are stacking columns

Teams.head()

In [ ]:
# Convert years into decade bins and creating dummy variables
def assign_decade(year):
    if year < 1920:
        return 1910
    elif year >= 1920 and year <= 1929:
        return 1920
    elif year >= 1930 and year <= 1939:
        return 1930
    elif year >= 1940 and year <= 1949:
        return 1940
    elif year >= 1950 and year <= 1959:
        return 1950
    elif year >= 1960 and year <= 1969:
        return 1960
    elif year >= 1970 and year <= 1979:
        return 1970
    elif year >= 1980 and year <= 1989:
        return 1980
    elif year >= 1990 and year <= 1999:
        return 1990
    elif year >= 2000 and year <= 2009:
        return 2000
    elif year >= 2010:
        return 2010
 

In [ ]:
Teams['decade_label'] = Teams['yearID'].apply(assign_decade)
decade_df = pd.get_dummies(Teams['decade_label'], prefix='decade')

Teams = pd.concat([Teams, decade_df], axis=1) # the axis=1 statement indicates we are stacking columns

Teams.head()

In [ ]:
# Create new features for Runs per Game and Runs Allowed per Game
Teams = Teams.assign(R_per_game = Teams['R'] / Teams['G'] ,
                     RA_per_game = Teams['RA'] / Teams['G'])

# Merge the mlb_rpg from years back to Teams
Teams = pd.merge(Teams, year.loc[:, "mlb_rpg"].to_frame(), # the .to_frame() is a technically detail we can discuss later
                 how="left", left_on="yearID", right_index=True) # note how we are indicating the join columns

# Drop unnecessary columns
Teams.drop(['yearID','year_label','decade_label'], axis=1, inplace=True)

Teams.head()

## Predicting wins

We are now finished prepping the data and will now show you how to build a model to predict wins in a season.

In [ ]:
# Create scatter plots for runs per game vs. wins and runs allowed per game vs. wins
fig = plt.figure(figsize=(12, 6))

ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

ax1.scatter(Teams['R_per_game'], Teams['W'], c='blue')
ax1.set_title('Runs per Game vs. Wins')
ax1.set_ylabel('Wins')
ax1.set_xlabel('Runs per Game')

ax2.scatter(Teams['RA_per_game'], Teams['W'], c='red')
ax2.set_title('Runs Allowed per Game vs. Wins')
ax2.set_xlabel('Runs Allowed per Game')

plt.show()


In [ ]:
Teams.corr()["W"] # Produces the correlations with the column "W"

## Clustering the variables

Next we will produce clusters in the data from the predictors

In [ ]:
attributes = ['G','R','AB','H','2B','3B','HR',
              'BB','SO','SB','RA','ER','ERA','CG',
              'SHO','SV','IPouts','HA','HRA','BBA',
              'SOA','E','DP','FP','era_1','era_2',
              'era_3','era_4','era_5','era_6','era_7',
              'era_8','decade_1910','decade_1920',
              'decade_1930','decade_1940','decade_1950',
              'decade_1960','decade_1970','decade_1980',
              'decade_1990','decade_2000','decade_2010',
              'R_per_game','RA_per_game', 'mlb_rpg']

data_attributes = Teams[attributes]

# Print the first rows of `df`
data_attributes.head()

In [ ]:
# Import necessary modules from `sklearn` 
from sklearn.cluster import KMeans
from sklearn import metrics

# Create silhouette score dictionary
s_score_dict = {}
for i in range(2,11):
    km = KMeans(n_clusters=i, random_state=1)
    l = km.fit_predict(data_attributes)
    s_s = metrics.silhouette_score(data_attributes, l)
    s_score_dict[i] = [s_s]

# Print out `s_score_dict`
plt.plot(s_score_dict.keys(), s_score_dict.values());

In [ ]:
# Create K-means model and determine euclidian distances for each data point
kmeans_model = KMeans(n_clusters=6, random_state=1)
distances = kmeans_model.fit_transform(data_attributes)

# Create scatter plot using labels from K-means model as color
labels = kmeans_model.labels_

plt.scatter(distances[:,2], distances[:,5], c=labels)
plt.title('Kmeans Clusters')

plt.show()

In [ ]:
# Add labels from K-means model to `df` DataFrame and attributes list
Teams = Teams.assign(labels = labels)

decade_df = pd.get_dummies(Teams['labels'], prefix='cluster')

Teams = pd.concat([Teams, decade_df], axis=1)

Teams.drop('labels', axis=1, inplace=True)

for i in range(6):
    attributes.append('cluster_{}'.format(i))

# Print the first rows of `Teams`
Teams.head()

In [ ]:
attributes

### A few words about sklearn

Sklearn provides a consistent modeling framework (API?) that can produce very much *drag-n-drop* functionality.

This is the basic framework

```python
# Import your favorite algorith, e.g. linear_model, cluster
from sklearn.algorithm_family import FavAlgo

# Create an instance of the algorithm
model = FavAlgo(param_0, param_1, ...)

# Fit the model 
results = model.fit(X, y)

# Get results from the model
results.predict(X)
results.other_neat_stuff(...) # such as coefficients, errors, scores, fitted values, etc...
```

In addition to the models, sklearn provides a beginning to end framework for modeling, for example data processing and splitting, feature selection, cross validation, and metrics.  For example, here is how you would split your data into a training versus a test data sets.

In [ ]:
# Import `train_test_split` from `sklearn.model_selection`
from sklearn.model_selection import train_test_split

X, y = Teams[attributes], Teams.W

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, random_state=1)

In [ ]:
# Import `LinearRegression` from `sklearn.linear_model`
from sklearn.linear_model import LinearRegression

# Import `mean_absolute_error` from `sklearn.metrics`
from sklearn.metrics import mean_absolute_error

# Create Linear Regression model, fit model, and make predictions
lr = LinearRegression(normalize=True)
lr.fit(X_train, y_train)
# Determine mean absolute error
mae_train = mean_absolute_error(y_train, lr.predict(X_train))
mae_test = mean_absolute_error(y_test, lr.predict(X_test))

# Print `mae`
print("Training mean error:", mae_train)
print("Testing mean error:", mae_test)

print("Training R2:", lr.score(X_train, y_train))
print("Testing R2:", lr.score(X_test, y_test))

In [ ]:
{attributes[i]: round(lr.coef_[i], 3) for i in range(len(attributes))} 

In [ ]:
plt.scatter(lr.predict(X), y, alpha=.15)

We can apply regularization to the linear model below if you so wish.  Just another example of the *drag-n-drop* nature of the process.

In [ ]:
# Import `RidgeCV` from `sklearn.linear_model`
from sklearn.linear_model import RidgeCV

# Create Ridge Linear Regression model, fit model, and make predictions
rrm = RidgeCV(alphas=(0.01, 0.1, 1.0, 10.0), normalize=True)
rrm.fit(X_train, y_train)
predictions_rrm = rrm.predict(X_test)

# Determine mean absolute error
mae_rrm = mean_absolute_error(y_test, predictions_rrm)
print(mae_rrm)
print(rrm.score(X_test, y_test))

In [ ]:
{attributes[i]: (round(lr.coef_[i], 3), round(rrm.coef_[i], 3)) for i in range(len(attributes))}

## Further Reading

For a general introduction to all of the tools.
[Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook)

For a deeper introduction to Pandas + others.
[Python for Data Analysis](https://github.com/wesm/pydata-book)

For a deeper introduction to Sklearn.
[Introduction to Machine Learning with Python: A Guide for Data Scientist](https://github.com/amueller/introduction_to_ml_with_python)

For Python quickly.
[A Whirlwind Tour of Python](https://jakevdp.github.io/WhirlwindTourOfPython/)

Talk to me if you'd like to borrow any of these - I have digital copies.